<a href="https://colab.research.google.com/github/VesleAnne/Diploma/blob/main/Notebooks/%D0%A1%D0%BE%D0%B7%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%B0%D0%B9%D0%BF%D0%BB%D0%B0%D0%B9%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import random

import pickle

In [ ]:
# Load data
df = pd.read_csv('/content/drive/MyDrive/МФТИ/вкр/dataset.csv').dropna()

In [ ]:
df.tail()

,question,answer,label
15592,🖼 Ссылка на jivo media_guid_0849b75c-8c30-44a0...,Здравствуйте! Чем могу помочь? 👋,2
15593,"Получилось, ура!\nСпасибо!",asdf,1
15594,чтобы отобразить вес объекта с названием,13,2
15595,а где конкретно там нужно вставить этот запрос ?,Здравствуйте! Чем могу помочь? 👋,1
15596,🖼 Ссылка на jivo media_guid_95049188-f120-4bdd...,Здравствуйте! Чем могу помочь? 👋,1


In [ ]:
# Define  base model
base_model = 'sentence-transformers/msmarco-bert-base-dot-v5'
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModel.from_pretrained("/content/drive/MyDrive/МФТИ/вкр/BERT_2/LSTM_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Создание словаря, который будет содержать токенизатор и модель
tokenizer_and_model = {
    'tokenizer': tokenizer,
    'model': model
}

In [ ]:
# Сохранение словаря в файл пикл
with open("LSTM.pkl", "wb") as f:
    pickle.dump(tokenizer_and_model, f)

In [ ]:
# Загрузка объединенного токенизатора и модели из файла пикл
with open("/content/drive/MyDrive/МФТИ/вкр/Пайплайн/lLSTM.pkl", "rb") as f:
    loaded_data = pickle.load(f)

# Получение токенизатора и модели из загруженных данных
tokenizer = loaded_data['tokenizer']
model = loaded_data['model']


In [ ]:
def get_embeddings_from_dataset(dataset, tokenizer, model, max_length):
    embeddings = []
    for q in dataset['question']:
        # Tokenize input sequence
        encoded_q = tokenizer(q, return_tensors='pt', truncation=True, max_length=max_length)
        encoded_q = {key: value for key, value in encoded_q.items()}
        with torch.no_grad():
            # Forward pass through the model
            q_embedding = model(**encoded_q).pooler_output
        embeddings.append(q_embedding)
    return embeddings

In [ ]:
# Define maximum sequence length
max_length = 256

# Getting embeddings from the dataset
embeddings = get_embeddings_from_dataset(df, tokenizer, model, max_length)

In [ ]:
def find_similar_answers(question, dataset, tokenizer, model, embeddings, top_n=1):
    # Tokenizing the input question
    encoded_input = tokenizer(question, return_tensors='pt')

    # Getting the question embeddings from the model
    with torch.no_grad():
        question_embedding = model(**encoded_input).pooler_output.cpu()  # Move to CPU
        # Computing the cosine similarity between the question and all questions in the dataset with added random noise
    similarities = []
    for q_embedding in embeddings:
        similarity = cosine_similarity(question_embedding, q_embedding.cpu())  # Move to CPU
        similarities.append(similarity)

    # Getting indexes of the most similar questions
    top_indices = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:top_n]

    # Returning the most similar answers and their corresponding proximity probabilities
    similar_answers = [(dataset['answer'][idx], similarities[idx]) for idx in top_indices]
    return similar_answers


In [ ]:
question = "Зачем связывать тип объекта и бизнес-процесс?"

similar_answers = find_similar_answers(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer: Для создания бизнес-процесса необходимо сделать следующее:
1. Зайти в реестр бизнес-процессов
2. Нажать на кнопку “Создать”
3. Далее откроется карточка бизнес-процесса. Необходимо заполнить поля “Наименование” и “Идентификатор”
4. Нажать на кнопку “Сохранить”

Подробнее в статье 


Similarity: [[1.]]
----------------------


In [ ]:
question = "Позови оператора!"

similar_answers = find_similar_answers(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer: По вопросам авторизации и прохождения курса – обращайтесь по адресу academy@greendatasoft.ru.
Similarity: [[0.9786608]]
----------------------


In [ ]:
question = "Не работает приложение"

similar_answers = find_similar_answers(question, df, tokenizer, model, embeddings)
for answer, similarity in similar_answers:
    print("Answer:", answer)
    print("Similarity:", similarity)
    print("----------------------")

Answer: 🖼 Ссылка на jivo media_guid_dfa126b6-f60b-4786-ba77-b475bfd0e708
Similarity: [[0.9758463]]
----------------------
